##✅ Passo a passo – Mapa com controle temporal (Folium + GeoPandas)

###1. Monte o ambiente no Google Colab


In [38]:
# Instale se necessário
!pip install geopandas folium

###2. Monte o Google Drive e leia o arquivo .gpkg

In [39]:
from google.colab import drive
import geopandas as gpd
import pandas as pd

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
# Caminho para o arquivo
caminho = '/content/drive/MyDrive/análise entorno sinistro/pontos_sinistro_atrop_MunCampinas_22-25_wgs84.gpkg'

# Lê o arquivo (assumindo que a camada se chama 'pontos_sinistro_atrop_22-25_wgs84')
gdf = gpd.read_file(caminho)

###3. Converta a coluna data_sinis para formato de data

In [41]:
# Se houver espaço, padronize os nomes
gdf.columns = gdf.columns.str.strip().str.lower()

# Converte a coluna para datetime
gdf['data_sinis'] = pd.to_datetime(gdf['data_sinis'], errors='coerce', dayfirst=True)

###4. Crie o GeoJSON com a estrutura exigida pelo TimestampedGeoJson

In [42]:
features = []
for _, row in gdf.iterrows():
    if pd.notnull(row['data_sinis']):
        features.append({
            'type': 'Feature',
            'geometry': row['geometry'].__geo_interface__,
            'properties': {
                'time': row['data_sinis'].isoformat(),
                'popup': f"Data: {row['data_sinis'].date()}<br>Tipo: {row['tipo_acide']}"
            }
        })

geojson = {
    'type': 'FeatureCollection',
    'features': features
}


##5. Construa o mapa interativo

####Versão diária acumulativa

In [ ]:
import folium
from folium.plugins import TimestampedGeoJson

# Centraliza no estado de SP (ajuste se quiser)
m = folium.Map(location=[-23.55, -46.63], zoom_start=10)

TimestampedGeoJson(
    geojson,
    transition_time=200,
    period='P1D',  # passo de 1 dia
    add_last_point=True,
    auto_play=False,
    loop=False,
    max_speed=1,
    loop_button=True,
    date_options='YYYY-MM-DD',
    time_slider_drag_update=True
).add_to(m)

m.save('mapa_temporal_diario_acumulado.html')


####Versão mensal não acumulativa

In [ ]:
import folium
from folium.plugins import TimestampedGeoJson

# Centraliza no centroide dos dados
centro = gdf.geometry.unary_union.centroid
m = folium.Map(location=[centro.y, centro.x], zoom_start=12)

# Gera as features com estilo e pop-up detalhado
features = []
for _, row in gdf.iterrows():
    popup = f"""
    <b>Data:</b> {row['data_sinis'].date()}<br>
    <b>Tipo de Acidente:</b> {row.get('tipo_acide', '')}<br>
    <b>Gravidade:</b> {row.get('gravidade_', '')}<br>
    <b>Município:</b> {row.get('municipio', '')}<br>
    <b>Veículos:</b> {', '.join([str(row.get(f'tp_veicu_{i}', '')) for i in range(1, 7) if row.get(f'tp_veicu_{i}')])}
    """

    feature = {
        'type': 'Feature',
        'geometry': row['geometry'].__geo_interface__,
        'properties': {
            'time': row['data_sinis'].isoformat(),
            'popup': popup,
            'style': {
                'color': 'black',
                'fillColor': 'red',
                'weight': 0.3,
                'radius': 3,
                'fillOpacity': 0.7
            }
        }
    }
    features.append(feature)

geojson = {
    'type': 'FeatureCollection',
    'features': features
}

# Adiciona camada com tempo: mostra apenas os pontos do dia
TimestampedGeoJson(
    geojson,
    period='P1M',                # passo de 1 mês
    duration='P1M',              # mostra apenas 1 mês por vez
    auto_play=False,
    loop=False,
    loop_button=True,
    time_slider_drag_update=True,
    date_options='YYYY-MM'
).add_to(m)

# Salva o mapa como HTML
m.save('mapa_temporal_mensal.html')


<ipython-input-8-6f4e54506e43>:5: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  centro = gdf.geometry.unary_union.centroid


####Versão mensal não acumulativa de calor

In [47]:
from folium.plugins import HeatMapWithTime
import folium

# Garante que a data está no formato correto
gdf = gdf.dropna(subset=['data_sinis', 'geometry'])
gdf['ano_mes'] = gdf['data_sinis'].dt.to_period('M')

# Ordena os períodos únicos
periodos = sorted(gdf['ano_mes'].unique())

# Agrupa as coordenadas por mês
heat_data = []
for periodo in periodos:
    grupo = gdf[gdf['ano_mes'] == periodo]
    coords = [[row.geometry.y, row.geometry.x] for row in grupo.itertuples() if row.geometry is not None]
    heat_data.append(coords)

#Define gradiente manual aproximado de viridis
#viridis_gradient = {
#    0.0: "#fde725",   # amarelo claro (baixa densidade)
#    0.7: "#5ec962",   # verde
#    0.9: "#3b528b",   # azul escuro
#    1.0: "#440154"    # roxo escuro (alta densidade)
#}

# Cria o mapa centralizado no centroide
centro = gdf.geometry.unary_union.centroid
m = folium.Map(location=[centro.y, centro.x], zoom_start=12)

# Adiciona o mapa de calor com tempo
HeatMapWithTime(
    data=heat_data,
    index=[str(p) for p in periodos],  # exemplo: '2022-01'
    auto_play=False,
    max_opacity=0.65,
    radius=20,
    use_local_extrema=True,
#    gradient=viridis_gradient

).add_to(m)

# Salva o resultado
m.save('heatmap_temporal_mensal11.html')


<ipython-input-47-dafc54b7a530>:27: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  centro = gdf.geometry.unary_union.centroid


**Observação:** Foi testada a aplicação de um gradiente de cores inspirado na paleta viridis utilizando o parâmetro gradient do plugin HeatMapWithTime do Folium. Embora o código para essa personalização tenha sido mantido com # (comentado) para referência futura, o resultado visual não ficou satisfatório. O gradiente não se comporta como esperado e a limitação do próprio plugin impede uma representação fiel das cores, tornando essa abordagem inviável para o tipo de mapa desejado.

####Versão Mapa de Calor com Seletor de Mês (manual)

In [11]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import folium
from folium.plugins import HeatMap

# Agrupa os dados por ano e mês (caso ainda não tenha feito)
gdf['ano_mes'] = gdf['data_sinis'].dt.to_period('M')
meses_disponiveis = sorted(gdf['ano_mes'].unique().astype(str))

# Cria dropdown
dropdown = widgets.Dropdown(
    options=meses_disponiveis,
    description='Mês:',
    layout={'width': '50%'}
)

# Função para gerar e salvar o mapa
def atualizar_mapa(mes_str):
    clear_output(wait=True)
    display(dropdown)

    grupo = gdf[gdf['ano_mes'].astype(str) == mes_str]
    coords = [[row.geometry.y, row.geometry.x] for row in grupo.itertuples() if row.geometry is not None]

    centro = gdf.geometry.unary_union.centroid
    mapa = folium.Map(location=[centro.y, centro.x], zoom_start=12)

    HeatMap(
        coords,
        radius=15,
        max_opacity=0.8,
        use_local_extrema=True
    ).add_to(mapa)

    # Salva o mapa
    nome_arquivo = f'/content/drive/MyDrive/heatmap_{mes_str}.html'
    mapa.save(nome_arquivo)
    print(f"Mapa salvo em: {nome_arquivo}")

# Conecta o dropdown ao evento de mudança
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        atualizar_mapa(change['new'])

dropdown.observe(on_change)
display(dropdown)


Dropdown(description='Mês:', index=18, layout=Layout(width='50%'), options=('2022-01', '2022-02', '2022-03', '…

<ipython-input-11-63e71e811dd2>:25: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  centro = gdf.geometry.unary_union.centroid


Mapa salvo em: /content/drive/MyDrive/heatmap_2023-07.html


Ideia: Um controle dentro do próprio mapa HTML, como um dropdown embutido no canto superior esquerdo ou direito do mapa em que o usuário poderia interagir diretamente no mapa, sem depender da interface do Google Colab.

❌ Por que isso não funciona direto com Folium:
O folium gera mapas com base no Leaflet.js, mas não permite incluir interações JavaScript customizadas facilmente via Python. Ele é limitado a plugins prontos.

Para esse tipo de controle embutido (como dropdown no mapa), você precisa de:

🛠️ Opções para ter um seletor dentro do mapa
🔹 1. HTML + Leaflet.js puro (JavaScript)
Você cria um arquivo HTML com:

Um <select> (dropdown) no canto do mapa

Um script JS que troca a camada de calor conforme a seleção

🔹 2. Dash + dash-leaflet
Você cria um app Python com:

Dropdown no layout (fora ou embutido no mapa)

Muda dinamicamente o conteúdo do HeatMap

🔹 3. Streamlit + folium + streamlit-folium (meio termo)
Streamlit permite exibir um mapa e interagir com filtros de forma mais amigável que Colab.

✅ Alternativa simples:
Um arquivo HTML com:

Múltiplas camadas de heatmap (uma por mês)

Um controle de camadas (tipo checkbox ou dropdown) no canto do mapa para trocar de mês

Isso funciona dentro do mapa, mesmo em um único HTML.



####Versão Mapa com seletor de mês embutido no mapa

In [13]:
import folium
from folium.plugins import HeatMap
from shapely.geometry import Point

# Garante que a coluna de data e geometria estão corretas
gdf = gdf.dropna(subset=['data_sinis', 'geometry'])
gdf['ano_mes'] = gdf['data_sinis'].dt.to_period('M')
meses_disponiveis = sorted(gdf['ano_mes'].unique().astype(str))

# Centraliza o mapa no centroide dos dados
centro = gdf.geometry.unary_union.centroid
m = folium.Map(location=[centro.y, centro.x], zoom_start=12)

# Gera uma camada de heatmap para cada mês
for mes in meses_disponiveis:
    grupo = gdf[gdf['ano_mes'].astype(str) == mes]
    coords = [[geom.y, geom.x] for geom in grupo.geometry if isinstance(geom, Point)]

    if coords:  # Só adiciona camada se houver pontos
        layer = folium.FeatureGroup(name=mes, show=False)
        HeatMap(
            coords,
            radius=15,
            max_opacity=0.8,
            use_local_extrema=True
        ).add_to(layer)
        layer.add_to(m)

# Adiciona controle de camadas
folium.LayerControl(collapsed=False).add_to(m)

# Salva o HTML
m.save('heatmap_por_mes_com_selector.html')


<ipython-input-13-41e5391d9fd7>:11: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  centro = gdf.geometry.unary_union.centroid


📦 **Bloco Mapa de Calor com Seletor de Mês (manual)**


 🧾 **Explicação: Por que não dá pra combinar os dois no Google Colab**

❌ Limitações do Google Colab:

* O Colab **não permite interações JavaScript diretamente embutidas** no mapa HTML com `folium`.
* O controle `HeatMapWithTime` é **renderizado em HTML**, com JavaScript rodando no navegador. Já o dropdown com `ipywidgets` é **Python puro**, processado no backend do notebook.
* Ou seja: **não é possível colocar os dois controles (barra temporal + dropdown) no mesmo mapa interativo diretamente pelo Folium no Colab**.

✅ Como seria possível combinar os dois:

Para ter **ambos os controles (barra + seletor)** no **mesmo mapa com calor**:

* Você precisaria montar uma **aplicação em HTML/JavaScript** usando **Leaflet.js** e bibliotecas como:

  * `leaflet-timeline-slider` ou `leaflet-daterangepicker`
  * Controle de camadas e eventos personalizados com JS

Ou então, criar uma aplicação interativa com:

💡 Alternativa: `Dash + dash-leaflet`

* Você cria um app web em Python.
* Um `DatePickerRange` ou `Dropdown` permite selecionar meses.
* Um controle com botão play/pause pode ser programado à parte.
* Você renderiza heatmaps customizados com base na seleção.

---

✅ Em resumo:

| Caminho                    | Dá pra fazer no Colab? | Permite os dois controles? | Complexidade |
| -------------------------- | ---------------------- | -------------------------- | ------------ |
| `folium + HeatMapWithTime` | ✅ Sim                  | ❌ Só barra temporal        | Baixa        |
| `folium + widgets`         | ✅ Sim                  | ❌ Só seletor manual        | Baixa        |
| `Dash + dash-leaflet`      | ❌ Não (só local/web)   | ✅ Sim, ambos               | Alta         |
| `Leaflet.js + JS`          | ❌ Não                  | ✅ Sim, ambos               | Alta         |


